In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
import xgboost as xgb
import catboost as cab
import time
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
import gc

In [2]:
events = pd.read_pickle('data/events.pkl')
session = pd.read_pickle('data/session.pkl')
submission = pd.read_csv('data/submission.csv')

In [3]:
new_user = set(submission.user_id_hash)-((set(events.user_id_hash)|set(session.user_id_hash))&set(submission.user_id_hash))

In [4]:
len(new_user)

208

In [3]:
lgbm_1 = np.load('./prediction/lgbm_prediction_1_0320_meta.npy')

In [4]:
lgbm_2 = np.load('./prediction/lgbm_prediction_2_0320_meta.npy')

In [6]:
xgb_1 = np.load('./prediction/xgb_prediction_1_0306.npy')
xgb_2 = np.load('./prediction/xgb_meta_2_0306.npy')

In [7]:
cb_1 = np.load('./prediction/cb_prediction_1_0302.npy')
cb_2 = np.load('./prediction/cb_prediction_2_0302.npy')

In [8]:
nn_1 = np.load('./prediction/nn_prediction_1.npy')

In [5]:
meta_1 = np.load('./prediction/lgbm_prediction_1_0320_meta.npy')
meta_2 = np.load('./prediction/lgbm_prediction_2_0320_meta.npy')

In [11]:
label1_df = pd.DataFrame({'lgbm_prediction_label_1': lgbm_1,
                          'xgb_prediction_label_1': xgb_1,
                          'cb_prediction_label_1': cb_1,
                          'meta_prediction_label_1': meta_1})
label2_df = pd.DataFrame({'lgbm_prediction_label_2': lgbm_2,
                          'xgb_prediction_label_2': xgb_2,
                          'cb_prediction_label_2': cb_2,
                          'meta_prediction_label_2': meta_2})

In [12]:
label1_df.corr()

,lgbm_prediction_label_1,xgb_prediction_label_1,cb_prediction_label_1,meta_prediction_label_1
lgbm_prediction_label_1,1.000000,0.949051,0.984162,0.961825
xgb_prediction_label_1,0.949051,1.000000,0.929473,0.941940
cb_prediction_label_1,0.984162,0.929473,1.000000,0.954332
meta_prediction_label_1,0.961825,0.941940,0.954332,1.000000


In [13]:
label2_df.corr()

,lgbm_prediction_label_2,xgb_prediction_label_2,cb_prediction_label_2,meta_prediction_label_2
lgbm_prediction_label_2,1.000000,0.800757,0.997047,0.797236
xgb_prediction_label_2,0.800757,1.000000,0.792313,0.978732
cb_prediction_label_2,0.997047,0.792313,1.000000,0.789472
meta_prediction_label_2,0.797236,0.978732,0.789472,1.000000


In [6]:
submission['user_purchase_binary_7_days'] = meta_1
submission['user_purchase_binary_14_days'] = meta_2

In [24]:
submission.user_purchase_binary_7_days.min()

9.735509434020754e-07

In [7]:
submission.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.000163,0.000264
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.000023,0.000054
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.000030,0.000053
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.000364,0.000877
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.000015,0.000019


In [8]:
submission.to_csv('submission0320.csv', index=False)

In [16]:
!kaggle competitions list -s predict-in-app

No competitions found


In [17]:
!kaggle competitions submit predict-in-app-purchase -f submission0320.csv -m 'lgbm check'

100%|██████████████████████████████████████| 32.6M/32.6M [00:01<00:00, 32.7MB/s]
Successfully submitted to Predict in-app purchases

In [7]:
len(prediction1)

3881197

In [5]:
submission['user_purchase_binary_7_days'] = prediction1

ValueError: Length of values does not match length of index

In [6]:
submission['user_purchase_binary_14_days'] = prediction2

In [8]:
lgbm_prediction_label_1 = np.load('./prediction/lgbm_prediction_1_attr_0227.npy')
lgbm_prediction_label_2 = np.load('./prediction/lgbm_prediction_2_attr_0227.npy')

In [9]:
xgb_prediction_label_1 = np.load('./prediction/xgb_prediction_label_1.npy')
xgb_prediction_label_2 = np.load('./prediction/xgb_prediction_label_2.npy')

In [6]:
cb_prediction_label_1 = np.load('cb_prediction_label_1.npy')
cb_prediction_label_2 = np.load('cb_prediction_label_2.npy')

In [10]:
label1 = (lgbm_prediction_label_1+xgb_prediction_label_1+lgbm_prediction_label_1_0220)/3

label2 = (lgbm_prediction_label_2+xgb_prediction_label_2+lgbm_prediction_label_2_0220)/3

In [18]:
label1 = (lgbm_prediction_label_1+xgb_prediction_label_1)/2

label2 = (lgbm_prediction_label_2+xgb_prediction_label_2)/2

In [14]:
label1 = lgbm_prediction_label_1
# label2 = lgbm_prediction_label_2

In [20]:
label1

array([0.00307692, 0.00316145, 0.0026637 , ..., 0.00243856, 0.00791641,
       0.00335474])

In [8]:
submission.columns

Index(['user_id_hash', 'user_purchase_binary_7_days',
       'user_purchase_binary_14_days'],
      dtype='object')

In [17]:
submission.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.003077,0.002703
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.003161,0.001595
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.002664,0.001321
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.004754,0.003565
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.007721,0.011801


In [14]:
submission.user_purchase_binary_7_days = label1
submission.user_purchase_binary_14_days = label2

In [15]:
submission.to_csv('submission0227_attr_lgbm_xgb.csv', index=False)

In [16]:
!kaggle competitions submit predict-in-app-purchase -f submission0227_attr_lgbm_xgb.csv -m 'with attr and xgb'

100%|██████████████████████████████████████| 32.0M/32.0M [00:00<00:00, 37.8MB/s]
Successfully submitted to Predict in-app purchases

In [11]:
label1_df = pd.DataFrame({'lgbm_prediction_label_1': prediction1,
                          'xgb_prediction_label_1': xgb_prediction_label_1,
                          'lgbm_prediction_label_1_attr': lgbm_prediction_label_1})

In [12]:
label1_df.corr()

,lgbm_prediction_label_1,xgb_prediction_label_1,lgbm_prediction_label_1_attr
lgbm_prediction_label_1,1.000000,0.835893,0.979003
xgb_prediction_label_1,0.835893,1.000000,0.807863
lgbm_prediction_label_1_attr,0.979003,0.807863,1.000000


In [13]:
label2_df = pd.DataFrame({'lgbm_prediction_label_2': lgbm_prediction_label_2,
                          'xgb_prediction_label_2': xgb_prediction_label_2,
                          'lgbm_prediction_label_2_0220': lgbm_prediction_label_2_0220})

In [14]:
label2_df.corr()

,lgbm_prediction_label_2,xgb_prediction_label_2,lgbm_prediction_label_2_0220
lgbm_prediction_label_2,1.000000,0.822367,0.997461
xgb_prediction_label_2,0.822367,1.000000,0.819360
lgbm_prediction_label_2_0220,0.997461,0.819360,1.000000


In [3]:
lgbm_prediction_label_1_0220 = np.load('lgbm_prediction_label_1_0220.npy')
lgbm_prediction_label_2_0220 = np.load('lgbm_prediction_label_2_0220.npy')

In [4]:
submission['user_purchase_binary_7_days'] = lgbm_prediction_label_1_0220
submission['user_purchase_binary_14_days'] = lgbm_prediction_label_2_0220

In [15]:
submission['user_purchase_binary_7_days'] = label1
submission['user_purchase_binary_14_days'] = label2

In [14]:
submission.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.000141,0
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.000151,0
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.000064,0
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.000491,0
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.000167,0


In [17]:
submission.to_csv('submission0220_user_encoding_3blending.csv', index=False)

In [18]:
!kaggle competitions submit predict-in-app-purchase -f submission0220_user_encoding_3blending.csv -m 'try user encoding'

100%|██████████████████████████████████████| 32.2M/32.2M [00:02<00:00, 13.7MB/s]
Successfully submitted to Predict in-app purchases

In [19]:
!kaggle datasets init -p model_data -t

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: unrecognized arguments: -t
